# Review Rec Bot

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from loguru import logger
import json

import sys
sys.path.insert(0, '..')

In [3]:
import os

from tqdm.notebook import tqdm
import pandas as pd
from llama_index.core.response.notebook_utils import display_source_node

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

# Arguments

In [6]:
from src.run.args import RunInputArgs

ARGS = RunInputArgs(
    EXPERIMENT_NAME="Review Rec Bot - Yelp Review Rec Bot",
    RUN_NAME="020_add_context_relevancy_metric",
    RUN_DESCRIPTION="""
# Add Context Relevancy Metric

## Changelog
""",
    TESTING=False,
    LOG_TO_MLFLOW=True,
    OBSERVABILITY=True,
    RECREATE_INDEX=False,
    RECREATE_RETRIEVAL_EVAL_DATASET=False,
    RECREATE_RESPONSE_EVAL_DATASET=False,
    DEBUG=False,
)

ARGS

{
  "EXPERIMENT_NAME": "Review Rec Bot - Yelp Review Rec Bot",
  "RUN_NAME": "020_add_context_relevancy_metric",
  "RUN_DESCRIPTION": "\n# Add Context Relevancy Metric\n\n## Changelog\n",
  "TESTING": false,
  "DEBUG": false,
  "OBSERVABILITY": true,
  "LOG_TO_MLFLOW": true,
  "RECREATE_RETRIEVAL_EVAL_DATASET": false,
  "RECREATE_RESPONSE_EVAL_DATASET": false,
  "RECREATE_INDEX": false
}

# Load config

In [7]:
from src.run.cfg import RunConfig

In [8]:
cfg = RunConfig()
cfg.init(ARGS)

2024-08-10 15:43:52.183 | INFO     | src.run.cfg:init:146 - Starting Observability server with Phoenix...
INFO:phoenix.config:📋 Ensuring phoenix working directory: /home/dvquys/.phoenix
2024-08-10 15:43:56.702 | INFO     | src.run.cfg:init:164 - Setting up MLflow experiment Review Rec Bot - Yelp Review Rec Bot - run 020_add_context_relevancy_metric...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


2024-08-10 15:43:56.911 | INFO     | src.run.cfg:init:173 - Notebook-generated artifacts are persisted at data/020_add_context_relevancy_metric


In [9]:
cfg

/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


{
  "args": {
    "EXPERIMENT_NAME": "Review Rec Bot - Yelp Review Rec Bot",
    "RUN_NAME": "020_add_context_relevancy_metric",
    "RUN_DESCRIPTION": "\n# Add Context Relevancy Metric\n\n## Changelog\n",
    "TESTING": false,
    "DEBUG": false,
    "OBSERVABILITY": true,
    "LOG_TO_MLFLOW": true,
    "RECREATE_RETRIEVAL_EVAL_DATASET": false,
    "RECREATE_RESPONSE_EVAL_DATASET": false,
    "RECREATE_INDEX": false
  },
  "app_name": "review_rec_bot",
  "storage_context_persist_dp": "data/018_finetuned_embedding_reindex/storage_context",
  "db_collection": "review_rec_bot__018_finetuned_embedding_reindex__huggingface____data_finetune_embedding_finetuned_model",
  "notebook_cache_dp": "data/020_add_context_relevancy_metric",
  "data_fp": "../data/yelp_dataset/sample/sample_100_biz/denom_review.parquet",
  "llm_cfg": {
    "llm_provider": "togetherai",
    "llm_model_name": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    "embedding_provider": "huggingface",
    "embedding_model_name

## Set up logger to collect additional info

In [10]:
collect_fp = f"{cfg.notebook_cache_dp}/collect.log"
logger.add(collect_fp, filter=lambda record: "[COLLECT]" in record['message'], mode='w')

1

# Load input data

In [11]:
data = pd.read_parquet(cfg.data_fp)
data = data.assign(
    biz_categories=lambda df: df['biz_categories'].str.split(', '),
    date=lambda df: df['date'].dt.strftime('%Y-%m-%dT%H:%M:%S')
)
logger.info(f"[COLLECT] {len(data)=}")
data.iloc[0]

2024-08-10 15:43:57.076 | INFO     | __main__:<module>:6 - [COLLECT] len(data)=5240


review_id                                      L0jv8c2FbpWSlfNC6bbUEA
user_id                                        bFPdtzu11Oi0f92EAcjqmg
business_id                                    IDtLPgUrqorrpqSLdfMhZQ
review_stars                                                        5
useful                                                              0
funny                                                               0
cool                                                                0
text                What a great addition to the Funk Zone!  Grab ...
date                                              2016-10-13T22:50:47
biz_name                                         Helena Avenue Bakery
biz_address                                     131 Anacapa St, Ste C
biz_city                                                Santa Barbara
biz_state                                                          CA
biz_postal_code                                                 93101
biz_latitude        

In [12]:
input_data = data
if ARGS.TESTING:
    input_data = data[:20]
logger.info(f"[COLLECT] {len(input_data)=}")

2024-08-10 15:43:57.105 | INFO     | __main__:<module>:4 - [COLLECT] len(input_data)=5240


In [13]:
input_data.columns

Index(['review_id', 'user_id', 'business_id', 'review_stars', 'useful',
       'funny', 'cool', 'text', 'date', 'biz_name', 'biz_address', 'biz_city',
       'biz_state', 'biz_postal_code', 'biz_latitude', 'biz_longitude',
       'biz_stars', 'biz_review_count', 'biz_is_open', 'biz_attributes',
       'biz_categories', 'biz_hours'],
      dtype='object')

# Prepare documents

In [14]:
from llama_index.core import Document

documents = []
embedding_visible_metadata = ["review_stars", "biz_name", "biz_address", "biz_city", "biz_state", "biz_categories"]
excluded_embed_metadata_keys = [k for k in input_data.columns if k not in embedding_visible_metadata]

for i, row in tqdm(input_data.iterrows(), total=len(input_data)):
    record = row.to_dict()
    text = record['text']
    metadata = {k: v for k, v in record.items() if k not in ('text')}
    
    doc = Document(
        text=text,
        metadata=metadata,
        excluded_embed_metadata_keys=excluded_embed_metadata_keys,
        excluded_llm_metadata_keys=excluded_embed_metadata_keys
    )
    documents.append(doc)

logger.info(f"[COLLECT] {len(documents)=}")

  0%|          | 0/5240 [00:00<?, ?it/s]

2024-08-10 15:43:57.657 | INFO     | __main__:<module>:20 - [COLLECT] len(documents)=5240


#### Check document embedding text

In [15]:
from llama_index.core.schema import MetadataMode

In [16]:
document = documents[0]
print(document.get_content(metadata_mode=MetadataMode.EMBED))

review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries']

What a great addition to the Funk Zone!  Grab a bite, grab some tastings, life is good. Right next door to the Santa Barbara Wine Collective, in fact it actually shares the same tables.  We had a fabulous savory croissant.


# Set up LLM

In [17]:
llm, embed_model = cfg.setup_llm()

WARNI [sentence_transformers.SentenceTransformer] You try to use a model that was created with version 2.7.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [18]:
print(cfg.llm_cfg.model_dump_json(indent=2))

{
  "llm_provider": "togetherai",
  "llm_model_name": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
  "embedding_provider": "huggingface",
  "embedding_model_name": "./data/finetune_embedding/finetuned_model",
  "embedding_model_dim": 768,
  "ollama__host": "192.168.100.14",
  "ollama__port": 11434
}


In [19]:
from llama_index.core import Settings
Settings.embed_model = embed_model
Settings.llm = llm

# Vector Store

In [20]:
import qdrant_client
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

from src.run.orchestrator import RunOrchestrator

In [21]:
qdrantdb = qdrant_client.QdrantClient(
    host="localhost",
    port=6333
)
aqdrantdb = qdrant_client.AsyncQdrantClient(
    host="localhost",
    port=6333
)

RunOrchestrator.setup_db(cfg, qdrantdb)

db_collection = qdrantdb.get_collection(cfg.db_collection)
vector_store = QdrantVectorStore(
    client=qdrantdb,
    collection_name=cfg.db_collection,
    aclient=aqdrantdb,
    enable_hybrid=False,
    prefer_grpc=True
)

2024-08-10 15:44:01.409 | INFO     | src.run.orchestrator:setup_db:37 - Use existing Qdrant collection: review_rec_bot__018_finetuned_embedding_reindex__huggingface____data_finetune_embedding_finetuned_model
WARNI [llama_index.vector_stores.qdrant.base] Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.


# Index Embeddings

In [22]:
import time
import pickle
from multiprocessing import set_start_method
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.node_parser import SemanticSplitterNodeParser
# from llama_index.core.node_parser import SentenceSplitter

In [23]:
chunker = SemanticSplitterNodeParser
chunker_cfg = {
    "buffer_size": 1,
    "breakpoint_percentile_threshold": 95,
    "embed_model": embed_model
}

# chunker = SentenceSplitter
# chunker_cfg = {
#     "chunk_size": 512,
#     "chunk_overlap": 10
# }

In [24]:
t0 = time.perf_counter()
# TODO: TO understand the differences between points_count and indexed_vector_counts.
# Here indexed_vector_counts = 0
db_collection_count = db_collection.points_count

if db_collection_count > 0 and ARGS.RECREATE_INDEX == False:
    logger.info(f"Loading Storage Context from {cfg.storage_context_persist_dp}...")
    docstore = SimpleDocumentStore.from_persist_dir(persist_dir=cfg.storage_context_persist_dp)
    storage_context = StorageContext.from_defaults(
        docstore=docstore,
        vector_store=vector_store
    )
    nodes = list(docstore.docs.values())
else:
    logger.info(f"Creating new DB index...")
    from llama_index.core.extractors import TitleExtractor
    from llama_index.core.ingestion import IngestionPipeline, IngestionCache

    # create the pipeline with transformations
    pipeline = IngestionPipeline(
        transformations=[
            chunker(**chunker_cfg),
            embed_model,
        ],
        vector_store = vector_store
    )

    num_workers = None
    # TODO: I can get the preprocessing completed much quicker if I push the initial steps through a multiprocessing loop and then separately create the embeddings using the built-in batching already provided in the SentenceTransformer encode method.
    # Ref: https://github.com/run-llama/llama_index/issues/10104#issuecomment-1899401584
    # Currently setting num_workers leads to code simple hang
    # Ref: https://github.com/run-llama/llama_index/issues/10104
    # num_workers = os.cpu_count() - 1
    # os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    # set_start_method("spawn", force=True)  # it hangs without this line
    logger.info(f"Running Ingestion Pipeline with {num_workers=}...")
    nodes = await pipeline.arun(documents=documents, num_workers=num_workers, show_progress=True)

    # Define Docstore as an abstraction on top of nodes to easily manage (e.g. get node by id)
    docstore = SimpleDocumentStore()
    await docstore.async_add_documents(nodes)
    storage_context = StorageContext.from_defaults(
        docstore=docstore,
        vector_store=vector_store
    )
    logger.info(f"Persisting Storage Context to {cfg.storage_context_persist_dp}...")
    storage_context.persist(cfg.storage_context_persist_dp)

t1 = time.perf_counter()

2024-08-10 15:44:01.530 | INFO     | __main__:<module>:7 - Loading Storage Context from data/018_finetuned_embedding_reindex/storage_context...


In [25]:
logger.info(f"Indexing {len(documents)} documents into VectorStoreIndex took {t1 - t0:,.0f}s")
logger.info(f"[COLLECT] {len(nodes)=}")

2024-08-10 15:44:09.190 | INFO     | __main__:<module>:1 - Indexing 5240 documents into VectorStoreIndex took 8s
2024-08-10 15:44:09.190 | INFO     | __main__:<module>:2 - [COLLECT] len(nodes)=10219


# Analyze Chunks

In [26]:
for i, node in enumerate(nodes[:5]):
    print(f"\n\n==========Node {i+1}============")
    print(node.metadata)
    print(node.get_text())



==========Node 1============
{'review_id': 'L0jv8c2FbpWSlfNC6bbUEA', 'user_id': 'bFPdtzu11Oi0f92EAcjqmg', 'business_id': 'IDtLPgUrqorrpqSLdfMhZQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-10-13T22:50:47', 'biz_name': 'Helena Avenue Bakery', 'biz_address': '131 Anacapa St, Ste C', 'biz_city': 'Santa Barbara', 'biz_state': 'CA', 'biz_postal_code': '93101', 'biz_latitude': 34.4144445, 'biz_longitude': -119.6906718, 'biz_stars': 4.0, 'biz_review_count': 389, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly':

# Construct Retriever

In [27]:
from llama_index.core.retrievers import VectorIndexRetriever

In [28]:
print(cfg.retrieval_cfg.model_dump_json(indent=2))

{
  "retrieval_top_k": 10,
  "retrieval_dense_top_k": 5,
  "retrieval_sparse_top_k": 15,
  "retrieval_similarity_cutoff": null,
  "rerank_top_k": 5,
  "rerank_model_name": "BAAI/bge-reranker-large"
}


In [29]:
# configure retriever
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)
vector_retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=cfg.retrieval_cfg.retrieval_dense_top_k,
    # sparse_top_k=cfg.retrieval_cfg.retrieval_sparse_top_k,
)

In [30]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=cfg.retrieval_cfg.retrieval_sparse_top_k,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

bm25_retrieval_results = bm25_retriever.retrieve("Nashville, grocery")
for result in bm25_retrieval_results:
    display_source_node(result, source_length=1000, show_source_metadata=True)

DEBUG [bm25s] Building index from IDs objects


**Node ID:** 85d45fd0-53db-4bac-82b0-f7eceea27be5<br>**Similarity:** 3.2125637531280518<br>**Text:** Convenient grocery store. A bit ghetto, and not because of it's location at all.<br>**Metadata:** {'review_id': '-kdFaer2K56NJCGNadTe5w', 'user_id': 'UYgAC9mvMYc12Ufc5KVUnA', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 3, 'useful': 1, 'funny': 1, 'cool': 0, 'date': '2010-10-27T15:07:05', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': None, 'Ambience': None, 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': None, 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': None, 'GoodForMeal': None, 'HappyHour': None, 'HasTV': None, 'Music': None, 'NoiseLevel': None, 'OutdoorSeating': None, 'RestaurantsAttire': None, 'RestaurantsDelivery': None, 'RestaurantsGoodForGroups': None, 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': None, 'RestaurantsTableService': None, 'RestaurantsTakeOut': None, 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': None}, 'biz_categories': ['Food', 'Grocery'], 'biz_hours': {'Friday': '6:0-1:0', 'Monday': '6:0-1:0', 'Saturday': '6:0-1:0', 'Sunday': '6:0-1:0', 'Thursday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'Wednesday': '6:0-1:0'}}<br>

**Node ID:** 4d1a9d64-f9da-4eef-af8e-00be4b2b2a56<br>**Similarity:** 3.1722183227539062<br>**Text:** A good grocery store, just requires a car to get to (or bus). This used to be somewhere they shuttled local university students from Penn to come get groceries; not sure if they still do that.

There's almost always a good deal to be had, fresh produce and meat, and feels like a truly suburban grocery store. Lines are long at times, but the only real problem is the distance from the city if you don't have a car.<br>**Metadata:** {'review_id': 'hKTTE9mOS1tXI6gDbDU_Mw', 'user_id': 'nnwBdqGHIAJQ5QX9lHOtrQ', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 3, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2009-01-20T16:11:51', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': None, 'Ambience': None, 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': None, 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': None, 'GoodForMeal': None, 'HappyHour': None, 'HasTV': None, 'Music': None, 'NoiseLevel': None, 'OutdoorSeating': None, 'RestaurantsAttire': None, 'RestaurantsDelivery': None, 'RestaurantsGoodForGroups': None, 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': None, 'RestaurantsTableService': None, 'RestaurantsTakeOut': None, 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': None}, 'biz_categories': ['Food', 'Grocery'], 'biz_hours': {'Friday': '6:0-1:0', 'Monday': '6:0-1:0', 'Saturday': '6:0-1:0', 'Sunday': '6:0-1:0', 'Thursday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'Wednesday': '6:0-1:0'}}<br>

**Node ID:** ea019f4c-6507-4f1b-a59e-87ecfa981cc6<br>**Similarity:** 3.0762240886688232<br>**Text:** This grocery store SUCKS!  But it's the closest one to my house, so I go there for things that I need fast.  The produce is the worst possible.  I don't think the prices are very good, but then again it seems that all grocery stores in Philly are outrageously priced.  Employees are the slowest moving in the world!<br>**Metadata:** {'review_id': 'krw5FlAWXS16NkkqViK-wg', 'user_id': 'B-d6QdtkkoLHCyIyuT6y4A', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 2, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2011-05-12T20:29:04', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': None, 'Ambience': None, 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': None, 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': None, 'GoodForMeal': None, 'HappyHour': None, 'HasTV': None, 'Music': None, 'NoiseLevel': None, 'OutdoorSeating': None, 'RestaurantsAttire': None, 'RestaurantsDelivery': None, 'RestaurantsGoodForGroups': None, 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': None, 'RestaurantsTableService': None, 'RestaurantsTakeOut': None, 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': None}, 'biz_categories': ['Food', 'Grocery'], 'biz_hours': {'Friday': '6:0-1:0', 'Monday': '6:0-1:0', 'Saturday': '6:0-1:0', 'Sunday': '6:0-1:0', 'Thursday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'Wednesday': '6:0-1:0'}}<br>

**Node ID:** 75c476c7-332e-40c1-8533-903afc4bf975<br>**Similarity:** 3.0317909717559814<br>**Text:** Free parking garage.  Eat in grocery store or deli type restaurant....everything is by the pound.<br>**Metadata:** {'review_id': 'vQxUaNanrYGZ_X3qCiIC3g', 'user_id': 'btAiOvMIfAbP5Sc5fAR8zA', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2017-07-08T18:07:49', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'beer_and_wine'", 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': True, 'casual': True}", 'BYOB': None, 'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}", 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': 'False', 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': True, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}", 'HappyHour': 'True', 'HasTV': 'False', 'Music': "{'dj': False, 'background_music': False, 'no_music': False, 'jukebox': False, 'live': False, 'video': False, 'karaoke': False}", 'NoiseLevel': "u'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': "u'no'", 'WheelchairAccessible': 'True', 'WiFi': "'free'"}, 'biz_categories': ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', 'Diners'], 'biz_hours': {'Friday': '7:0-17:0', 'Monday': '7:0-17:0', 'Saturday': '7:0-17:0', 'Sunday': '7:0-17:0', 'Thursday': '7:0-17:0', 'Tuesday': '7:0-17:0', 'Wednesday': '7:0-17:0'}}<br>

...

**Node ID:** 2ed73c5c-6b33-4e0e-ba76-34195afce9cb<br>**Similarity:** 2.6478774547576904<br>**Text:** If you're looking for great tasting, high quality Italian food then this is the place for you. They make their own bread that is to die for. They have the best manicotti that I've ever had. Yum! There isn't a single bad thing on the menu. They also offer a unique Italian grocery selection with imported and domestic Italian products that are very hard to find at regular grocery stores. Support small business and give this place a try!<br>**Metadata:** {'review_id': '7jSEUBirxk_ghx8OI9myQg', 'user_id': 'DIXNEOmHO6D-zOM0X9fNEg', 'business_id': '0bPLkL0QhhPO5kt1_EXmNQ', 'review_stars': 5, 'useful': 2, 'funny': 0, 'cool': 1, 'date': '2014-01-03T22:13:26', 'biz_name': "Zio's Italian Market", 'biz_address': '2575 E Bay Dr', 'biz_city': 'Largo', 'biz_state': 'FL', 'biz_postal_code': '33771', 'biz_latitude': 27.9161159, 'biz_longitude': -82.7604608, 'biz_stars': 4.5, 'biz_review_count': 100, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': 'False', 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': False, 'breakfast': False}", 'HappyHour': None, 'HasTV': 'True', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'False', 'RestaurantsAttire': "u'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'False', 'RestaurantsPriceRange2': '1', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'no'"}, 'biz_categories': ['Food', 'Delis', 'Italian', 'Bakeries', 'Restaurants'], 'biz_hours': {'Friday': '10:0-20:0', 'Monday': '10:0-18:0', 'Saturday': '10:0-20:0', 'Sunday': None, 'Thursday': '10:0-20:0', 'Tuesday': '10:0-20:0', 'Wednesday': '10:0-20:0'}}<br>

**Node ID:** 0928245a-1313-49cb-89d1-51cc2e8f6416<br>**Similarity:** 2.554818630218506<br>**Text:** A must stop in Nashville<br>**Metadata:** {'review_id': 'BbJRG7HkGx0S3SuvvpVvIQ', 'user_id': '26nVu2FZTrYt3PEMXLCR0A', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2013-09-28T04:48:16', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': 'False', 'Caters': 'False', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': 'False', 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': None, 'HappyHour': None, 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'None', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': None, 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food'], 'biz_hours': {'Friday': '8:0-0:0', 'Monday': '8:0-23:0', 'Saturday': '8:0-0:30', 'Sunday': '8:0-23:0', 'Thursday': '8:0-23:0', 'Tuesday': '8:0-23:0', 'Wednesday': '8:0-23:0'}}<br>

**Node ID:** b8ec3423-1742-4463-a2fb-eb5033b46447<br>**Similarity:** 2.446824073791504<br>**Text:** This was my fiancé and Is first stop in Nashville.<br>**Metadata:** {'review_id': 'iTxg0SJaJhpn4H3lfL1wZg', 'user_id': 'OMMwV1eDH4bduIkbRsMMRw', 'business_id': 'tMkwHmWFUEXrC9ZduonpTg', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2019-07-07T05:47:41', 'biz_name': 'The Green Pheasant', 'biz_address': '215 1st Ave S', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1598858, 'biz_longitude': -86.7731974, 'biz_stars': 4.0, 'biz_review_count': 161, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'full_bar'", 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': True, 'casual': False}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'False', 'BusinessAcceptsBitcoin': 'False', 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': True, 'street': False, 'validated': True, 'lot': False, 'valet': True}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'False', 'GoodForMeal': "{'dessert': None, 'latenight': None, 'lunch': None, 'dinner': True, 'brunch': None, 'breakfast': None}", 'HappyHour': 'True', 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'False', 'RestaurantsAttire': "u'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': None, 'RestaurantsReservations': 'True', 'RestaurantsTableService': 'True', 'RestaurantsTakeOut': 'None', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'free'"}, 'biz_categories': ['Restaurants', 'Japanese', 'Seafood'], 'biz_hours': {'Friday': '16:0-23:0', 'Monday': None, 'Saturday': '16:0-23:0', 'Sunday': None, 'Thursday': '16:0-22:0', 'Tuesday': None, 'Wednesday': '16:0-22:0'}}<br>

**Node ID:** f20d5c52-03be-413a-8db0-1fdc40d86e24<br>**Similarity:** 2.446824073791504<br>**Text:** Too bad it is the only one in Nashville.<br>**Metadata:** {'review_id': 'Txs_E7x0WnrTxCB7cT6_ig', 'user_id': '1YBdny5HItzC_GoOE_xevA', 'business_id': 'B2qqjCl_BNNnXvWWfpFiLA', 'review_stars': 1, 'useful': 1, 'funny': 0, 'cool': 0, 'date': '2017-10-01T17:47:10', 'biz_name': 'Pottery Barn Kids', 'biz_address': '4011 Hillsboro Pike, Ste 102', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37215', 'biz_latitude': 36.1036135, 'biz_longitude': -86.8162678, 'biz_stars': 1.5, 'biz_review_count': 13, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': None, 'Ambience': None, 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': None, 'ByAppointmentOnly': None, 'Caters': None, 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': None, 'GoodForMeal': None, 'HappyHour': None, 'HasTV': None, 'Music': None, 'NoiseLevel': None, 'OutdoorSeating': None, 'RestaurantsAttire': None, 'RestaurantsDelivery': None, 'RestaurantsGoodForGroups': None, 'RestaurantsPriceRange2': None, 'RestaurantsReservations': None, 'RestaurantsTableService': None, 'RestaurantsTakeOut': None, 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': None}, 'biz_categories': ['Shopping', 'Baby Gear & Furniture', 'Home Decor', 'Furniture Stores', 'Home & Garden'], 'biz_hours': {'Friday': '10:0-20:0', 'Monday': '0:0-0:0', 'Saturday': '10:0-20:0', 'Sunday': '12:0-18:0', 'Thursday': '10:0-20:0', 'Tuesday': '10:0-20:0', 'Wednesday': '10:0-20:0'}}<br>

**Node ID:** 7a93c39d-c68c-4bdd-96ac-188ad5b8a0ba<br>**Similarity:** 2.437446355819702<br>**Text:** This pathmark is the closest grocery store to my house, and I'm glad it's here because it saves me a lot of trips to Trader Joe's or Whole Foods. However, I'm always slightly annoyed when I leave here.<br>**Metadata:** {'review_id': '7C0BJ9L5pTjFmmx3HjBDKw', 'user_id': 'nnZc4F8ZykvL4mKRcH2M-A', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 2, 'useful': 1, 'funny': 0, 'cool': 0, 'date': '2015-04-22T01:34:37', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': None, 'Ambience': None, 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': None, 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': None, 'GoodForMeal': None, 'HappyHour': None, 'HasTV': None, 'Music': None, 'NoiseLevel': None, 'OutdoorSeating': None, 'RestaurantsAttire': None, 'RestaurantsDelivery': None, 'RestaurantsGoodForGroups': None, 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': None, 'RestaurantsTableService': None, 'RestaurantsTakeOut': None, 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': None}, 'biz_categories': ['Food', 'Grocery'], 'biz_hours': {'Friday': '6:0-1:0', 'Monday': '6:0-1:0', 'Saturday': '6:0-1:0', 'Sunday': '6:0-1:0', 'Thursday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'Wednesday': '6:0-1:0'}}<br>

In [31]:
# test_query = nodes[3].text
test_query = "Received confirmation of my order at 5 pm"
vector_results = vector_retriever.retrieve(test_query)
bm25_results = bm25_retriever.retrieve(test_query)

# TODO: Reconsider this test.
# Currently it aims to make sure the vector retriever, 
# when given a piece of text extracted from a node, 
# would return the original node.
# But what is the value of it?

# Check if the original node in top 10:
found = False
for vector_result in vector_results[:10]:
    target_id = bm25_results[0].id_
    if vector_result.id_ == target_id:
        found = True
        break
# assert found, "Can not find the original node in the top 10 vector results"
# assert bm25_results[0].id_ == vector_results[0].id_, "Different vector_retriever nodes vs bm25_retriever nodes"

In [32]:
from llama_index.core.retrievers import QueryFusionRetriever

# query_gen_prompt = """
# You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
# You should apply different techniques to create new strings. Here are some example techniques:
# - Technique 1 - Optimize for full-text search: Extract the key concepts and entities as keywords. This technique can generate multiple strings instead of just one. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  string 1 = "cold brew coffee", string 2 = "hanoi". Do not modify the keyword, keep the keyword exactly in the input query. Return each output string on a new line.
# - Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."
# - Technique 3 - Increase recall: Paraphase the original question into a new question but use different synonyms or broader concepts. Example input query: "How to find some good local grocery stores near Changi Singapore?" -> Expected output query: "Where to find a good mart selling Singaporean stuffs to the east of Singapore?"

# Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2, third technique 3. If run of of techniques then re-iterate from the start.

# Return one string on each line, related to the input query.

# Only return the strings. Never include the chosen technique.

# Input Query: {query}\n
# New strings:\n
# """

# query_gen_prompt = """
# You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
# You should apply different techniques to create new strings. Here are some example techniques:
# - Technique 1 - Optimize for full-text search: Rephrase the input query to contain only important keywords. Remove all stopwords and low information words. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  "cold brew coffee hanoi"
# - Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."
# - Technique 3 - Increase recall: Paraphase the original question into a new question but use different synonyms or broader concepts. Example input query: "How to find some good local grocery stores near Changi Singapore?" -> Expected output query: "Where to find a good mart selling Singaporean stuffs to the east of Singapore?"

# Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2, third technique 3. If run of of techniques then re-iterate from the start.

# Return one string on each line, related to the input query.

# Only return the strings. Never include the chosen technique.

# Input Query: {query}\n
# New strings:\n
# """

query_gen_prompt = """
You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
You should apply different techniques to create new strings. Here are some example techniques:
- Technique 1 - Optimize for full-text search: Rephrase the input query to contain only important keywords. Remove all stopwords and low information words. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  "cold brew coffee hanoi"
- Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."

Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2. If run of of techniques then re-iterate from the start.

Return one string on each line, related to the input query.

Only return the strings. Never include the chosen technique.

Input Query: {query}\n
New strings:\n
"""

from llama_index.llms.openai import OpenAI
llm = OpenAI(model=cfg.eval_cfg.response_eval_llm_model, **cfg.eval_cfg.response_eval_llm_model_config)

retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    llm=llm,
    similarity_top_k=cfg.retrieval_cfg.retrieval_top_k,
    num_queries=2,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    query_gen_prompt=query_gen_prompt
)

## Test retrieval

In [33]:
query = "What are some places to enjoy cold brew coffee"
retrieval_results = await retriever.aretrieve(query)
for node in retrieval_results:
    display_source_node(node, source_length=1000)
    print(node.get_content(metadata_mode=MetadataMode.EMBED))
    print("\n")

Generated queries:
cold brew coffee places
"I recently visited a charming café that specializes in cold brew coffee. The atmosphere was perfect for relaxing, and they had a fantastic selection of cold brews that were smooth and refreshing. Definitely a must-try for coffee lovers!"


**Node ID:** 64f06176-a59e-44be-b332-4507fc783195<br>**Similarity:** 0.06191025457811894<br>**Text:** If it's cold, come grab a coffee instead. They offer all the great styles and flavors of coffee as any other coffee house you'll find.<br>

review_stars: 5
biz_name: Mike's Ice Cream
biz_address: 129 2nd Ave N
biz_city: Nashville
biz_state: TN
biz_categories: ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food']

If it's cold, come grab a coffee instead. They offer all the great styles and flavors of coffee as any other coffee house you'll find.




**Node ID:** 4e272c53-d4a4-4e15-96f1-efa6c8d09e00<br>**Similarity:** 0.06187941713888418<br>**Text:** I'm a fan of dark roast but it's usually not available. My advice brew different flavors of coffee and keep it hot.<br>

review_stars: 4
biz_name: Wawa
biz_address: 3604 Chestnut St
biz_city: Philadelphia
biz_state: PA
biz_categories: ['Restaurants', 'Automotive', 'Delis', 'Gas Stations', 'Food', 'Coffee & Tea', 'Sandwiches', 'Convenience Stores']

I'm a fan of dark roast but it's usually not available. My advice brew different flavors of coffee and keep it hot.




...

review_stars: 4
biz_name: Caviar & Bananas
biz_address: 2031 Broadway
biz_city: Nashville
biz_state: TN
biz_categories: ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', 'Diners']

Need a bottle of wine? Check!
Need a cold brew coffee with just a splash of almond milk? Check!
Need eggs? Check!
Need a gift basket? Check!
Need a frozen Rose? (Oh didn't know I needed that but I DO!) check! 
Neeeeeed that chocolate? Check!
Need a place for an informal meet up. Check! 
Need a fast bar meal 8:00 pm? Check!




**Node ID:** 94eda5e8-53bc-40fb-bc9f-a97145b9626b<br>**Similarity:** 0.03009207275993712<br>**Text:** GREAT BREWS!!!
This place isn't always the first place folks think of when they decide to visit a brewery in Indy... but it should be!! It's on the outskirts of town, but well worth heading over to! They have some wonderful mixes/brews... and their many awards prove it! 

It has a nice sized tasting room that is comfy welcoming. Not to mention the ever changing chalk board with some awesome art work to fancy up their list of what is on tap. 

GREAT place and wonderful people!!!<br>

review_stars: 5
biz_name: Bier Brewery and Tap Room
biz_address: 5133 E 65th St
biz_city: Indianapolis
biz_state: IN
biz_categories: ['Food', 'Beer', 'Wine & Spirits', 'Breweries']

GREAT BREWS!!!
This place isn't always the first place folks think of when they decide to visit a brewery in Indy... but it should be!! It's on the outskirts of town, but well worth heading over to! They have some wonderful mixes/brews... and their many awards prove it! 

It has a nice sized tasting room that is comfy welcoming. Not to mention the ever changing chalk board with some awesome art work to fancy up their list of what is on tap. 

GREAT place and wonderful people!!!




**Node ID:** c8c8fac1-e207-4864-9ed7-c089814b8908<br>**Similarity:** 0.030090497737556562<br>**Text:** I don't always like their beers, mainly because I'm not necessarily a fan of all the styles of beer that they brew here, but I do recognize that this place has the best execution of the things they brew. In other words, while I may not like everything they serve, they do it all well. 

Some of my favorites here include their coffee porter and their pumpkin ale (the best pumpkin offering in the city). I recently also had their pumpkin porter which was amazing.<br>

review_stars: 4
biz_name: Bier Brewery and Tap Room
biz_address: 5133 E 65th St
biz_city: Indianapolis
biz_state: IN
biz_categories: ['Food', 'Beer', 'Wine & Spirits', 'Breweries']

I don't always like their beers, mainly because I'm not necessarily a fan of all the styles of beer that they brew here, but I do recognize that this place has the best execution of the things they brew. In other words, while I may not like everything they serve, they do it all well. 

Some of my favorites here include their coffee porter and their pumpkin ale (the best pumpkin offering in the city). I recently also had their pumpkin porter which was amazing.




## Retrieval Evaluation

### Synthetic

In [34]:
from src.run.eval import RetrievalEvaluator

In [35]:
retrieval_evaluator = RetrievalEvaluator()
retrieval_evaluator.generate_synthetic_dataset(cfg, nodes)

2024-08-10 15:44:17.026 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:79 - Loading retrieval_eval_nodes from data/018_finetuned_embedding_reindex/retrieval_synthetic_eval_dataset.json...
2024-08-10 15:44:17.027 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:85 - Loading existing synthetic retrieval eval dataset at data/018_finetuned_embedding_reindex/retrieval_synthetic_eval_dataset.json...


In [36]:
retrieval_evaluator.retrieval_eval_dataset.queries

{'faef7066-7118-4800-8f01-0ec95f94a48e': 'What are some restaurants with a lively atmosphere and diverse menus?',
 'd001d2b9-96d2-418c-9a57-6afb98b217df': 'What are some reliable shipping or delivery services that ensure items are received without damage?',
 '2946af06-0e68-47f7-bb5e-60abe195a754': 'What are some affordable parking options in Midtown Nashville?',
 '61eca289-7f41-45d6-b917-d1266401081d': 'What are some businesses or locations that have received negative reviews recently?',
 'b1fa3c41-6c98-4f54-a683-9f5de253c52c': 'What are some ice cream shops that offer a cozy indoor seating experience during winter?',
 '19336816-ba0f-4b4a-ace0-236c1ccce31d': 'What are some popular ice cream shops in Nashville that offer waffle cones?',
 '5bc1432c-2ab6-49f8-8404-9dd71569b22a': 'What are some sushi restaurants that offer a la carte options instead of tasting menus?',
 '2f8e7777-5163-429d-beeb-8b5c958e175a': 'What are some cafes known for their excellent breakfast sandwiches and coffee?',

In [37]:
retrieval_eval_results_df, retrieval_eval_results_full_df = await retrieval_evaluator.aevaluate(cfg, retriever)

Generated queries:
lively atmosphere diverse menus restaurants
"I visited a restaurant that had an amazing vibe and a menu filled with various cuisines. The ambiance was energetic, perfect for a night out with friends!"
Generated queries:
reliable shipping delivery services items received undamaged
"I recently used FastShip for my deliveries, and I was impressed with how careful they were. My items arrived in perfect condition, and the tracking updates were very helpful."
Generated queries:
affordable parking options midtown nashville
"I recently visited Midtown Nashville and found several affordable parking options. The lot on 21st Avenue was convenient and reasonably priced, making it easy to explore the area without worrying about parking fees."
Generated queries:
negative reviews businesses locations recent
Generated queries:
ice cream shops cozy indoor seating winter
"I visited Creamy Delights last winter and loved their cozy atmosphere. They have a great selection of ice cream fl

In [38]:
retrieval_eval_results_df

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,top_10_retrieval_eval,0.466667,0.31963,0.046667,0.466667,0.31963,0.077911


In [39]:
retrieval_eval_results_full_df

,query,expected_ids,retrieved_ids,hit_rate,mrr,precision,recall,ap,ndcg
0,What are some restaurants with a lively atmosp...,[ea2e9ef9-4b90-4206-a74e-b7893a5ff870],"[1414baba-3641-4493-a80b-e55d7b1f60c9, f9cb733...",0.0,0.000000,0.0,0.0,0.000000,0.000000
1,What are some reliable shipping or delivery se...,[be0d2e8a-6b8a-4f3e-a8e6-9ed84b10eb12],"[be0d2e8a-6b8a-4f3e-a8e6-9ed84b10eb12, d6d5509...",1.0,1.000000,0.1,1.0,1.000000,0.220092
2,What are some affordable parking options in Mi...,[ffba872c-1151-40d6-ada0-9913eeeddab8],"[91f654a2-d75a-4494-938c-6142cb707f8f, c39a1f9...",1.0,0.111111,0.1,1.0,0.111111,0.066254
3,What are some businesses or locations that hav...,[711bb479-40d3-40c1-9732-7af7e7a89678],"[15743426-f295-4af1-a255-73393a3d0abc, 39fe191...",0.0,0.000000,0.0,0.0,0.000000,0.000000
4,What are some ice cream shops that offer a coz...,[0fb0975d-58cf-4fff-a13a-aab1d7303b42],"[713a370f-5ad1-4eab-8373-260e662f49b9, b3ae99a...",0.0,0.000000,0.0,0.0,0.000000,0.000000
5,What are some popular ice cream shops in Nashv...,[e3fbd7a0-8a36-4e83-bea5-26d637b418d1],"[23cf94a5-5e07-4d3b-9bb2-cfbb239debd6, 4469202...",1.0,0.333333,0.1,1.0,0.333333,0.110046
6,What are some sushi restaurants that offer a l...,[fc891a15-7e11-4c45-8c2f-7f6c2761d178],"[fc891a15-7e11-4c45-8c2f-7f6c2761d178, ce7d64d...",1.0,1.000000,0.1,1.0,1.000000,0.220092
7,What are some cafes known for their excellent ...,[e7061230-bcfc-45d6-8fd2-b67f91d919af],"[72db6c1f-e733-463f-adc2-c0c47279781c, a128cbc...",1.0,0.333333,0.1,1.0,0.333333,0.110046
8,What are some credit card companies known for ...,[b6b645ce-fd9b-4c2d-9b7d-02e11c7d7a0e],"[5bd3bf52-ce4a-4ace-8657-162b3a9e5f90, 9da9c08...",0.0,0.000000,0.0,0.0,0.000000,0.000000
9,What are some restaurants that are known for t...,[788eccdf-896e-46f6-b7ff-07f9094e3252],"[8dab09e3-476d-4c03-855d-6994480cbe92, da0f526...",0.0,0.000000,0.0,0.0,0.000000,0.000000


#### Error Analysis

In [40]:
retrieval_eval_irrelevance_df = (
    retrieval_eval_results_full_df
    .loc[lambda df: df['hit_rate'].lt(1)]
    .sort_values(['hit_rate', 'mrr', 'precision', 'recall', 'ap', 'ndcg'])
)
with pd.option_context('display.max_colwidth', 400):
    display(retrieval_eval_irrelevance_df)

,query,expected_ids,retrieved_ids,hit_rate,mrr,precision,recall,ap,ndcg
0,What are some restaurants with a lively atmosphere and diverse menus?,[ea2e9ef9-4b90-4206-a74e-b7893a5ff870],"[1414baba-3641-4493-a80b-e55d7b1f60c9, f9cb7330-8ad2-4ec8-ab1e-51d4648d87d9, 036c4651-1f1f-43d3-84c2-c113a211b934, 24cf23fa-0301-4035-89da-6cffa1427d2d, fee84b08-7ef0-4771-b753-2f5284255c20, 0075c293-e929-4c10-a72e-fc0b290a17b6, 54f3981f-ba06-4011-8498-7cdb9e178347, 6cc5cdf7-9eac-4cfe-9d87-9f818c5ee95a, 3cd69e15-9cb1-490f-b259-006e2a3f0c35, f81a52ec-fab6-4eab-ac4f-a21120e5fe5b]",0.0,0.0,0.0,0.0,0.0,0.0
3,What are some businesses or locations that have received negative reviews recently?,[711bb479-40d3-40c1-9732-7af7e7a89678],"[15743426-f295-4af1-a255-73393a3d0abc, 39fe191b-f970-43a6-b9be-0052ede1cc31, 8bfe8d23-4c35-446d-b0b2-7f903b905b58, 08e2e3af-ba54-4a74-beb1-0b8d0eda49c4, 269888ad-ae0e-4954-be94-6a0b62d0f9ab, 775076b5-25b3-4660-b733-e09593005368, a38be06b-ab5c-4272-a687-0606fe52ca82, 75450b52-f77e-4a90-8735-05b61346fb82, ffc6fe75-7613-45f1-b05a-4e2a17f2053b, d81bdcca-c7d9-4b5c-80b1-366758e90108]",0.0,0.0,0.0,0.0,0.0,0.0
4,What are some ice cream shops that offer a cozy indoor seating experience during winter?,[0fb0975d-58cf-4fff-a13a-aab1d7303b42],"[713a370f-5ad1-4eab-8373-260e662f49b9, b3ae99aa-f6e9-4d67-8645-a36767664ef4, 71fe858d-ea96-4a59-9a85-4b9ff48913c1, 4673444a-74e3-4df2-a4af-de9ae0833a16, 3f9940fa-46b2-4ff3-af43-e3465bf2c0b9, b5036c30-7ff9-46ae-b541-f26c6ccc3151, b827068c-9d2e-4245-89d7-75501c7c5546, dbc2b7b5-f0a8-4145-91a7-c13cf90dd272, 1c8b565e-dd26-4ec5-9984-40a4aa695935, 88f348c4-8913-4723-b2a3-51914f7e904c]",0.0,0.0,0.0,0.0,0.0,0.0
8,What are some credit card companies known for good customer service and reliability?,[b6b645ce-fd9b-4c2d-9b7d-02e11c7d7a0e],"[5bd3bf52-ce4a-4ace-8657-162b3a9e5f90, 9da9c08a-d45f-4186-a05b-125879f02e46, 372f0b72-884b-4251-bf9b-65018d0ce554, 29672cbe-978c-47a0-8abd-dd6d5e2f101d, fa15cdfa-1e09-4c81-a4ee-105b5a8b1c0e, a31057c0-74d3-434a-9864-8289ab02c181, f609f739-1474-45ca-a0dd-80c589e2ff06, 98896ca2-436c-4744-9f7f-61f36f6a683f, d1b49663-579b-47c7-8023-8897caf975bb, 99d81751-e78f-4e37-8078-de526dedc8ae]",0.0,0.0,0.0,0.0,0.0,0.0
9,What are some restaurants that are known for their special occasion dining experiences?,[788eccdf-896e-46f6-b7ff-07f9094e3252],"[8dab09e3-476d-4c03-855d-6994480cbe92, da0f5261-0f3d-4549-936e-6d30a7a7f0e5, 4c0db334-61dd-4f92-91be-20a8deee381b, 6c3b3eeb-2259-4ffc-9985-39aea025ccaf, 18c40924-74db-41fb-8b3f-4231bf42abc6, 7e11b13b-e700-4406-b954-b54ed1a4cf85, 7315ae74-266f-4bb8-87e1-c17fe53208ee, fc3f08f4-90ca-4e55-8ad5-29d16d66c7d7, ace9d60f-ecd5-4ec1-9e6f-196b92e432a1, 8d63b62a-aefb-48a7-9dfa-5f785050370c]",0.0,0.0,0.0,0.0,0.0,0.0
10,What are some effective wellness centers that offer body contouring treatments?,[2e7763ea-76f7-49a4-88a0-e125c48063ae],"[01948087-d818-4067-8c03-0b4c93ff577c, 7fa72912-9e09-4ac4-b0e8-0e02281d9bb2, 080c075e-4cf1-4141-ba10-6beee4c23906, 3c5be912-7343-4df1-8d3b-01387291047c, aa5e8e64-5cb6-4069-92ff-533dc5c8bf80, 6fc65ebf-8295-415c-99d8-d0fa3da96907, c8a4a2c9-4e5b-4947-a344-4ec19e086525, b16d9a2e-279d-4e28-9db0-862d723542ab, a757e1ae-2466-43fb-9dcf-44a22f3d7020, c4149e29-59d5-4826-89e5-7a8a83ddc95c]",0.0,0.0,0.0,0.0,0.0,0.0
11,What are some ice cream shops known for their creamy flavors and homemade waffle cones?,[27309eac-72ce-4940-b4b0-9e449eb1c584],"[266a6a36-2d0d-478c-b182-274e8720d784, ef829e9c-247e-4c36-8901-bea9a686582d, c7e43381-4123-42b7-9a25-bea7e0030723, c39b6f74-15c1-46a8-8bf6-dc81e49d8c5f, 167fefe5-dbff-476f-8584-981fa5c53012, 3d21e9e5-00d0-4e47-86bf-7ef1cd8054d3, 2e2528fe-7df8-4cd5-af4c-7011ea51e779, 44692026-0ae1-4b27-8c88-1d37ca0f788c, e81565ef-3fb1-4cd8-9e0a-aeda9be1733a, 5235ef0e-1110-4685-9c93-4d4f3501c42a]",0.0,0.0,0.0,0.0,0.0,0.0
12,What are some popular shopping destinations for clothing and accessories?,[8c1de488-b6c0-46de-9709-c0b8b0175a91],"[2805ede1-1add-4e3a-ae24-0bc5df7b00f1, 93130751-4681-4948-a2dc-905b2f2f3685, b12cd437-2aa

In [41]:
for i, row in retrieval_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n============Error #{i+1}=============\n\n")
    print(f"Query:\n{row.query}\n")
    print(f"- Expected contexts:\n")
    expected_nodes = docstore.get_nodes(row.expected_ids)
    for node in expected_nodes:
        print(node.get_content(metadata_mode=MetadataMode.LLM))

    print(f"\n- Retrieved contexts:\n")
    retrieved_nodes = docstore.get_nodes(row.retrieved_ids)
    for i, node in enumerate(retrieved_nodes):
        print(f"+ Context #{i+1}:\n\n{node.get_content(metadata_mode=MetadataMode.LLM)}\n")



============Error #1=============


Query:
What are some restaurants with a lively atmosphere and diverse menus?

- Expected contexts:

review_stars: 3
biz_name: Lafitte's Landing Seafood House
biz_address: 1700 Lapalco Blvd
biz_city: Harvey
biz_state: LA
biz_categories: ['Restaurants', 'Soup', 'Seafood', 'Burgers']

LaFitte's is a new venture of A J Tusa, owner of New Orleans  Creole Cookery.

Large parking lot just off LAPLCO Blvd & Manhattan.

Nice decor, clean dining area, diverse menu.

We ordered stuffed lobster, mahi-mahi, & stuffed catfish platter.

Lively locals crowd, noisy, cold beer, can't hear music for din.  

"New" waitstaff is being trained.

- Retrieved contexts:

+ Context #1:

review_stars: 5
biz_name: Caviar & Bananas
biz_address: 2031 Broadway
biz_city: Nashville
biz_state: TN
biz_categories: ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', '

In [69]:
def check_retrieval_results(query):
    retrieval_results = retriever.retrieve(query)
    for node in retrieval_results:
        display_source_node(node, source_length=1000)
        print(node.get_content(metadata_mode=MetadataMode.EMBED))
        print("\n")
    return retrieval_results

query = "What are some good grocery stores in Nashville that offer curated items and unique local products?"
check_retrieval_results(query)

Generated queries:
grocery stores nashville curated items unique local products
"I recently visited a grocery store in Nashville that had an amazing selection of curated items and unique local products. The atmosphere was inviting, and I found some great local jams and artisanal cheeses that I couldn't resist!"


**Node ID:** ffba872c-1151-40d6-ada0-9913eeeddab8<br>**Similarity:** 0.04782608695652174<br>**Text:** Interior is gorgeous. Very clean and modern. It'll be great for a lot of Nashville and tourist Instagrammers to get their pose on. 

As far as groceries, lots of curated items from both Nashville and beyond. They have McClures pickles... being from Michigan I got super pumped about those. They also carry Jenis Ice Cream pints which are down right dangerous. Great selection of coffee, sweets, cheeses, charcuterie and wine.

The only reason it's not a full 5 is the parking situation. All the lots in midtown are super expensive or you need change for meters (it's 2017 I don't carry anything that's monetary unless it's a card). If you do meters, it's $1.50 per hour in that area.  Paid lots are about $8 an hour.<br>

review_stars: 4
biz_name: Caviar & Bananas
biz_address: 2031 Broadway
biz_city: Nashville
biz_state: TN
biz_categories: ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', 'Diners']

Interior is gorgeous. Very clean and modern. It'll be great for a lot of Nashville and tourist Instagrammers to get their pose on. 

As far as groceries, lots of curated items from both Nashville and beyond. They have McClures pickles... being from Michigan I got super pumped about those. They also carry Jenis Ice Cream pints which are down right dangerous. Great selection of coffee, sweets, cheeses, charcuterie and wine.

The only reason it's not a full 5 is the parking situation. All the lots in midtown are super expensive or you need change for meters (it's 2017 I don't carry anything that's monetary unless it's a card). If you do meters, it's $1.50 per hour in that area.  Paid lots ar

**Node ID:** 169c9a0b-3f26-43da-a5e0-1123d3fe4c82<br>**Similarity:** 0.03333333333333333<br>**Text:** Stumbled upon this hidden gem of a store when we went to have dinner at of of the local restaurants.. 

They have some very unique items.. High quality which mean you get what you pay for! They also have several local artist/products.. 

Customer service hands down excellent!<br>

review_stars: 5
biz_name: James Dant
biz_address: 5624 E Washington St
biz_city: Indianapolis
biz_state: IN
biz_categories: ['Fashion', 'Shopping', "Men's Clothing"]

Stumbled upon this hidden gem of a store when we went to have dinner at of of the local restaurants.. 

They have some very unique items.. High quality which mean you get what you pay for! They also have several local artist/products.. 

Customer service hands down excellent!




...

**Node ID:** 126a3b14-62d3-4dbb-9646-96e997bdeead<br>**Similarity:** 0.030309988518943745<br>**Text:** Hi, I'd like literally everything in this store. 

Seriously, if I had some serious cash in the bank, I'd purchase nearly every item in this store...if the size would fit my small body type. 

Honestly, this place is exactly what I'd like my apartment to look like. Woodsy, rustic, with a touch of urban. It appeals to the eyes, but even more the other senses, especially smell. The smells inside the store and the other products are my type of body products, spot on.

The biggest concern about the store is the price tag on most items. But, look at the quality and where it comes from and you can reason the price tag for a nice "treat yourself" moment every now and then. 

I particularly loved looking through the tons and tons of items throughout the store - body products, cologne, hair and beard grooming, clothes, shoes, bikes and gear, and the list goes on and on. 

If you're in the area, make the trip and give it a shot. They have a nice "Sales" section too if you don't want to break ...<br>

review_stars: 5
biz_name: James Dant
biz_address: 5624 E Washington St
biz_city: Indianapolis
biz_state: IN
biz_categories: ['Fashion', 'Shopping', "Men's Clothing"]

Hi, I'd like literally everything in this store. 

Seriously, if I had some serious cash in the bank, I'd purchase nearly every item in this store...if the size would fit my small body type. 

Honestly, this place is exactly what I'd like my apartment to look like. Woodsy, rustic, with a touch of urban. It appeals to the eyes, but even more the other senses, especially smell. The smells inside the store and the other products are my type of body products, spot on.

The biggest concern about the store is the price tag on most items. But, look at the quality and where it comes from and you can reason the price tag for a nice "treat yourself" moment every now and then. 

I particularly loved looking through the tons and tons of items throughout the store - body products, cologne, hair and beard grooming, clothes, shoes, bike

**Node ID:** 48daddaf-4457-4bc1-972b-dfafa3574652<br>**Similarity:** 0.029910714285714284<br>**Text:** At first, I defended this store because it's the only affordable grocery option for anyone south of South/north of Broad. But after several visits, I've realized it's okay for picking up a few items, but not a replacement for my regular stores (Shop Rite in South Philly/Jersey or Wegmans in Jersey). Yes, I still make regular grocery runs to Jersey, because the quality of the stores is so much better. I digress.<br>

review_stars: 2
biz_name: Pathmark
biz_address: 3021 Grays Ferry Ave
biz_city: Philadelphia
biz_state: PA
biz_categories: ['Food', 'Grocery']

At first, I defended this store because it's the only affordable grocery option for anyone south of South/north of Broad. But after several visits, I've realized it's okay for picking up a few items, but not a replacement for my regular stores (Shop Rite in South Philly/Jersey or Wegmans in Jersey). Yes, I still make regular grocery runs to Jersey, because the quality of the stores is so much better. I digress.




[NodeWithScore(node=TextNode(id_='ffba872c-1151-40d6-ada0-9913eeeddab8', embedding=None, metadata={'review_id': 'VZMjK-Moy-fN08mR2m9JUg', 'user_id': 'U5hfpmnqW65_u10C1i06Dw', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 4, 'useful': 4, 'funny': 1, 'cool': 0, 'date': '2017-06-23T12:16:51', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'beer_and_wine'", 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': True, 'casual': True}", 'BYOB': None, 'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}", 'BikeParking': 'True', 'BusinessAcc

# Response

In [43]:
from llama_index.core import get_response_synthesizer
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

from src.features.append_reference.custom_query_engine import ManualAppendReferenceQueryEngine

In [44]:
node_postprocessors = []

if cfg.retrieval_cfg.retrieval_similarity_cutoff is not None:
    node_postprocessors.append(SimilarityPostprocessor(similarity_cutoff=cfg.retrieval_cfg.retrieval_similarity_cutoff))

reranker = FlagEmbeddingReranker(model=cfg.retrieval_cfg.rerank_model_name, top_n=cfg.retrieval_cfg.rerank_top_k)
node_postprocessors.append(reranker)

response_synthesizer = get_response_synthesizer()
query_engine = ManualAppendReferenceQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=node_postprocessors,
)

## Test Query Engine

In [45]:
from llama_index.core.response.notebook_utils import (
    display_source_node,
    display_response,
)

In [46]:
# question = "where to find good cold brew coffee?"
question = "What are some gastropubs in Indianapolis that are known for their cocktails?"
response = query_engine.query(question)
display_response(response, show_source=True, show_metadata=True, show_source_metadata=True)

Generated queries:
gastropubs cocktails indianapolis
"I visited a fantastic gastropub in Indianapolis that had an impressive cocktail menu. The mixologists were creative, and I loved their unique twists on classic drinks."


**`Final Response:`** Based on the reviews, it seems that Village Tap Room is a popular gastropub in Indianapolis that is known for its craft cocktails. In fact, multiple reviewers have mentioned that the bar has an excellent selection of hand-crafted cocktails.


#### Referenced Paragraphs
**Village Tap Room**

> Jeremy, the bar tender, is fantastic. The cocktail and beer menu are tops. Food is also spot on.  (Review: `LRMgqHm7xRNEcHoRYzqlFQ`)

> This is NOT Chumleys. Excellent local craft beer selection.  Hand crafted cocktails. Great food. Friendly staff. Check this place out!! (Review: `6LsSfJ5yH2m05agEKfgPcw`)

> Much improved over Chumley's.  Nice selection of local craft beer.  Craft cocktails.  Food was very good.   (Review: `48NsBnBkWSRWtEtQKuPCGw`)

> Drinks, in my opinion, were trying too hard. I had the double cheeseburger and a side salad (which literally were comparable to any fast food establishment). My bf had the puffy tacos. Decent bar food, good atmosphere. One shining star is the homemade tater tots! (Review: `1favNOENNvFvVjnI3cin6Q`)

> I ordered the steak burger medium with chipotle salsa (I was experimenting). The steak burger was well done as was my friend who also ordered medium. The buns were the cheap, wonderbread, kind you get in the 12-pack for a block party bbq. The chipotle salsa appeared to just be some salsa from a common brand container. This all cost me $9, which was ridiculous. 

The restaurant was pretty much empty, but I'm not that surprised. Come here for the drink special before it goes out of business! (Review: `J3DrcnIoOSppEF3rwboxag`)

---

**`Source Node 1/5`**

**Node ID:** 9b726c4a-ac25-4bfb-812a-9270e1e28195<br>**Similarity:** 1.5322891473770142<br>**Text:** Jeremy, the bar tender, is fantastic. The cocktail and beer menu are tops. Food is also spot on.<br>**Metadata:** {'review_id': 'LRMgqHm7xRNEcHoRYzqlFQ', 'user_id': '_Lm_BbPwHcL7a5lm6PAJ4w', 'business_id': 'L_TT0BFmFwORAMaA83K54A', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-03-31T23:48:14', 'biz_name': 'Village Tap Room', 'biz_address': '838 Broad Ripple Ave', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46220', 'biz_latitude': 39.8699114, 'biz_longitude': -86.1435773, 'biz_stars': 2.5, 'biz_review_count': 23, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': True, 'thursday': True, 'sunday': False, 'saturday': True}", 'BikeParking': 'False', 'BusinessAcceptsBitcoin': 'False', 'BusinessAcceptsCreditCards': 'False', 'BusinessParking': None, 'ByAppointmentOnly': None, 'Caters': 'False', 'CoatCheck': 'False', 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'False', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}", 'HappyHour': 'False', 'HasTV': 'False', 'Music': "{'jukebox': True}", 'NoiseLevel': "u'average'", 'OutdoorSeating': 'False', 'RestaurantsAttire': "u'casual'", 'RestaurantsDelivery': 'False', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'False', 'Smoking': "u'no'", 'WheelchairAccessible': 'False', 'WiFi': "u'no'"}, 'biz_categories': ['Gastropubs', 'Cocktail Bars', 'Beer Bar', 'Bars', 'Restaurants', 'Nightlife'], 'biz_hours': None}<br>

...

**Node ID:** 22484feb-9784-457b-b589-9539ac0492c1<br>**Similarity:** 0.24414384365081787<br>**Text:** Drinks, in my opinion, were trying too hard. I had the double cheeseburger and a side salad (whic...<br>**Metadata:** {'review_id': '1favNOENNvFvVjnI3cin6Q', 'user_id': 'St5V7WhDsigYWasgQbupfw', 'business_id': 'L_TT0BFmFwORAMaA83K54A', 'review_stars': 3, 'useful': 2, 'funny': 1, 'cool': 1, 'date': '2016-03-06T15:19:10', 'biz_name': 'Village Tap Room', 'biz_address': '838 Broad Ripple Ave', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46220', 'biz_latitude': 39.8699114, 'biz_longitude': -86.1435773, 'biz_stars': 2.5, 'biz_review_count': 23, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': True, 'thursday': True, 'sunday': False, 'saturday': True}", 'BikeParking': 'False', 'BusinessAcceptsBitcoin': 'False', 'BusinessAcceptsCreditCards': 'False', 'BusinessParking': None, 'ByAppointmentOnly': None, 'Caters': 'False', 'CoatCheck': 'False', 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'False', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}", 'HappyHour': 'False', 'HasTV': 'False', 'Music': "{'jukebox': True}", 'NoiseLevel': "u'average'", 'OutdoorSeating': 'False', 'RestaurantsAttire': "u'casual'", 'RestaurantsDelivery': 'False', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'False', 'Smoking': "u'no'", 'WheelchairAccessible': 'False', 'WiFi': "u'no'"}, 'biz_categories': ['Gastropubs', 'Cocktail Bars', 'Beer Bar', 'Bars', 'Restaurants', 'Nightlife'], 'biz_hours': None}<br>

---

**`Source Node 5/5`**

**Node ID:** 622734c1-4764-4c12-9dbc-c08946d77186<br>**Similarity:** -0.03906284645199776<br>**Text:** I ordered the steak burger medium with chipotle salsa (I was experimenting). The steak burger was...<br>**Metadata:** {'review_id': 'J3DrcnIoOSppEF3rwboxag', 'user_id': 'bX_rCJvQq1mQ186PhNYYdg', 'business_id': 'L_TT0BFmFwORAMaA83K54A', 'review_stars': 2, 'useful': 1, 'funny': 0, 'cool': 1, 'date': '2016-09-09T12:22:08', 'biz_name': 'Village Tap Room', 'biz_address': '838 Broad Ripple Ave', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46220', 'biz_latitude': 39.8699114, 'biz_longitude': -86.1435773, 'biz_stars': 2.5, 'biz_review_count': 23, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': True, 'thursday': True, 'sunday': False, 'saturday': True}", 'BikeParking': 'False', 'BusinessAcceptsBitcoin': 'False', 'BusinessAcceptsCreditCards': 'False', 'BusinessParking': None, 'ByAppointmentOnly': None, 'Caters': 'False', 'CoatCheck': 'False', 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'False', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}", 'HappyHour': 'False', 'HasTV': 'False', 'Music': "{'jukebox': True}", 'NoiseLevel': "u'average'", 'OutdoorSeating': 'False', 'RestaurantsAttire': "u'casual'", 'RestaurantsDelivery': 'False', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'False', 'Smoking': "u'no'", 'WheelchairAccessible': 'False', 'WiFi': "u'no'"}, 'biz_categories': ['Gastropubs', 'Cocktail Bars', 'Beer Bar', 'Bars', 'Restaurants', 'Nightlife'], 'biz_hours': None}<br>

{'9b726c4a-ac25-4bfb-812a-9270e1e28195': {'review_id': 'LRMgqHm7xRNEcHoRYzqlFQ',
  'user_id': '_Lm_BbPwHcL7a5lm6PAJ4w',
  'business_id': 'L_TT0BFmFwORAMaA83K54A',
  'review_stars': 5,
  'useful': 0,
  'funny': 0,
  'cool': 0,
  'date': '2016-03-31T23:48:14',
  'biz_name': 'Village Tap Room',
  'biz_address': '838 Broad Ripple Ave',
  'biz_city': 'Indianapolis',
  'biz_state': 'IN',
  'biz_postal_code': '46220',
  'biz_latitude': 39.8699114,
  'biz_longitude': -86.1435773,
  'biz_stars': 2.5,
  'biz_review_count': 23,
  'biz_is_open': 0,
  'biz_attributes': {'AcceptsInsurance': None,
   'Alcohol': "u'none'",
   'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}",
   'BYOB': None,
   'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': True, 'thursday': True, 'sunday': False, 'saturday': True}",
   'BikeParking': 'False',
   'BusinessAccept

## Response Evaluation

In [47]:
from src.run.eval import ResponseEvaluator

In [48]:
response_evaluator = ResponseEvaluator()

### Synthetic

In [49]:
response_eval_documents, response_synthetic_eval_dataset = response_evaluator.generate_synthetic_dataset(cfg, documents)

2024-08-10 15:44:54.152 | INFO     | src.run.eval.response:generate_synthetic_dataset:99 - Sampling 30 documents for response evaluation...
2024-08-10 15:44:54.159 | INFO     | src.run.eval.response:generate_synthetic_dataset:155 - Loading existing synthetic response eval dataset at data/018_finetuned_embedding_reindex/response_synthetic_eval_dataset.json...


In [50]:
response_synthetic_eval_prediction_dataset = await response_synthetic_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=cfg.batch_size, show_progress=True
)

Batch processing of predictions:   0%|                                                                                                                                                                        | 0/16 [00:00<?, ?it/s]

Generated queries:
cocktail bars welcoming atmosphere philadelphia
"I visited a cocktail bar in Philadelphia that had such a warm and inviting vibe. The staff was friendly, and the ambiance made it a perfect spot for a night out with friends."
Generated queries:
wine bars new orleans great atmosphere
"I visited a fantastic wine bar in New Orleans that had an amazing ambiance and a great selection of wines. The atmosphere was perfect for a night out with friends."
Generated queries:
popular coffeehouses cozy atmosphere studying edmonton
"I recently visited a coffeehouse in Edmonton that had the perfect cozy vibe for studying. The ambiance was warm, with comfortable seating and plenty of outlets. It was a great place to enjoy a cup of coffee while getting some work done."
Generated queries:
ice cream shops variety flavors nashville
"I visited Sweet Creams in Nashville, and they had an amazing selection of flavors, from classic vanilla to unique options like lavender honey. Definitely a m

Batch processing of predictions:   0%|                                                                                                                                                                        | 0/14 [00:00<?, ?it/s]

Generated queries:
popular cafes nashville unique food coffee
"I visited a cafe in Nashville called The Coffee Shop, and they had an amazing selection of unique food options alongside their great coffee. The atmosphere was cozy and inviting!"
Generated queries:
wine bars new orleans great atmosphere highly-rated
"I visited a fantastic wine bar in New Orleans that had an amazing atmosphere. The ambiance was perfect for a night out, and the wine selection was top-notch!"
Generated queries:
gastropubs cocktails indianapolis
"I visited a fantastic gastropub in Indianapolis that had an impressive cocktail menu. The mixologists were creative, and I particularly enjoyed their signature drinks."
Generated queries:
bottomless mimosas brunch spots new orleans
"I recently visited a place called The Ruby Slipper in New Orleans, and they had an amazing brunch with bottomless mimosas. The atmosphere was lively, and the food was delicious!"
Generated queries:
ice cream shops unique flavors toppings n

Batch processing of predictions: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:24<00:00,  1.79s/it]


In [59]:
response_synthetic_mean_scores_df, response_synthetic_deep_eval_df = response_evaluator.evaluate_labelled_rag_dataset(
    response_synthetic_eval_dataset,
    response_synthetic_eval_prediction_dataset,
    dataset_name="synthetic",
    judge_model=cfg.eval_cfg.response_eval_llm_model,
    cache_dp=cfg.notebook_cache_dp
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [02:40<00:00,  5.36s/it]


In [60]:
response_synthetic_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,2.666667
mean_relevancy_score,0.866667
mean_faithfulness_score,0.833333
mean_context_relevancy_score,0.600000


In [61]:
response_synthetic_deep_eval_df

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
0,- What are some highly-rated Japanese restaura...,\nThere are several highly-rated Japanese rest...,1.0,3.0,1.0,0.625,[Fantastic Japanese restaurant downtown. Lovel...
1,- What are some Italian restaurants known for ...,\nIt seems that Andrew at the bar and Teresa a...,1.0,2.0,1.0,0.250,[Normally a great place to eat and great custo...
2,- What are some highly-rated wine bars in New ...,\nSome wine bars in New Orleans that are known...,1.0,2.0,1.0,0.625,[Wow!!! One of my top three new favorite place...
3,- What are some highly-rated wine bars in New ...,\nIf you're looking for a great wine bar exper...,1.0,2.0,1.0,0.500,[Decent little wine bar in New Orleans. Lots o...
4,- What are some ice cream shops that offer uni...,\nSome ice cream shops offer unique cone optio...,1.0,3.0,1.0,0.625,[!This ice cream shop is awesome! They pile it...
5,What are some popular coffeehouses in Edmonton...,\nThere are several popular coffeehouses in Ed...,1.0,3.0,1.0,0.625,[I went there yesterday afternoon for the firs...
6,What are some highly-rated sports bars in Wesl...,\nIf you're looking for highly-rated sports ba...,1.0,2.0,1.0,0.875,[This is the best sports bar and restaurant yo...
7,- What are some cocktail bars in Philadelphia ...,\nIt's clear that a vibrant and energetic atmo...,1.0,2.0,1.0,0.625,[The best new bar in Philly. Amazing drink men...
8,What are some sports bars in Indianapolis that...,\nIt sounds like a sports fan's paradise would...,1.0,2.0,1.0,0.625,[The best part obviously is the scenery and fr...
9,What are some popular wine bars in New Orleans...,\nIf you're looking for a unique wine-drinking...,1.0,3.0,1.0,0.625,[I love the atmosphere of Copper Vine! On top ...


#### Error Analysis

In [62]:
response_synthetic_eval_dataset_dict = dict()
for example in response_synthetic_eval_dataset.examples:
    response_synthetic_eval_dataset_dict[example.query] = {
        "reference_answer": example.reference_answer,
        "reference_contexts": example.reference_contexts,
    }

In [64]:
response_synthetic_deep_eval_df.sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score', 'context_relevancy_score'])

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
23,- What are some pizza places known for their q...,\nIt's worth noting that consistency is key wh...,0.0,2.0,0.0,0.000,[Domino's keep being you and I'll keep patroni...
25,- What are some mobile phone stores known for ...,"\nUnfortunately, I couldn't find any informati...",0.0,2.0,0.0,0.250,"[Okay, so after a couple of trips to this plac..."
24,- What are some highly-rated coffee shops in N...,"\nBased on the reviews, it seems that there ar...",0.0,2.0,0.0,0.500,"[The barista was rude, I asked her if the bake..."
20,- What are some sushi bars in Philadelphia wit...,"\nBased on the reviews, it seems that Royal Su...",0.0,3.0,0.0,0.625,"[Anyways. Food is great, management sucks. The..."
14,What are some sports bars in Indianapolis that...,\nIt seems that Twin Peaks has a unique atmosp...,1.0,2.0,0.0,0.250,[Twin peaks has been open for a couple of mont...
1,- What are some Italian restaurants known for ...,\nIt seems that Andrew at the bar and Teresa a...,1.0,2.0,1.0,0.250,[Normally a great place to eat and great custo...
22,- What are some gastropubs in Indianapolis tha...,\nIt seems that gastropubs in Indianapolis tha...,1.0,2.0,1.0,0.250,"[Jeremy, the bar tender, is fantastic. The coc..."
3,- What are some highly-rated wine bars in New ...,\nIf you're looking for a great wine bar exper...,1.0,2.0,1.0,0.500,[Decent little wine bar in New Orleans. Lots o...
2,- What are some highly-rated wine bars in New ...,\nSome wine bars in New Orleans that are known...,1.0,2.0,1.0,0.625,[Wow!!! One of my top three new favorite place...
7,- What are some cocktail bars in Philadelphia ...,\nIt's clear that a vibrant and energetic atmo...,1.0,2.0,1.0,0.625,[The best new bar in Philly. Amazing drink men...


In [66]:
CONTEXT_RELEVANCE_SCORE_THRESHOLD = 0.5

response_synthetic_eval_irrelevance_df = (
    response_synthetic_deep_eval_df
    .loc[lambda df: df['context_relevancy_score'].le(CONTEXT_RELEVANCE_SCORE_THRESHOLD)]
    .sort_values(['context_relevancy_score', 'relevancy_score', 'correctness_score', 'faithfulness_score'])
)

response_synthetic_eval_irrelevance_df

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
23,- What are some pizza places known for their q...,\nIt's worth noting that consistency is key wh...,0.0,2.0,0.0,0.00,[Domino's keep being you and I'll keep patroni...
25,- What are some mobile phone stores known for ...,"\nUnfortunately, I couldn't find any informati...",0.0,2.0,0.0,0.25,"[Okay, so after a couple of trips to this plac..."
14,What are some sports bars in Indianapolis that...,\nIt seems that Twin Peaks has a unique atmosp...,1.0,2.0,0.0,0.25,[Twin peaks has been open for a couple of mont...
1,- What are some Italian restaurants known for ...,\nIt seems that Andrew at the bar and Teresa a...,1.0,2.0,1.0,0.25,[Normally a great place to eat and great custo...
22,- What are some gastropubs in Indianapolis tha...,\nIt seems that gastropubs in Indianapolis tha...,1.0,2.0,1.0,0.25,"[Jeremy, the bar tender, is fantastic. The coc..."
24,- What are some highly-rated coffee shops in N...,"\nBased on the reviews, it seems that there ar...",0.0,2.0,0.0,0.50,"[The barista was rude, I asked her if the bake..."
3,- What are some highly-rated wine bars in New ...,\nIf you're looking for a great wine bar exper...,1.0,2.0,1.0,0.50,[Decent little wine bar in New Orleans. Lots o...
18,- What are some sports bars in Indianapolis th...,\nIf you're looking for a sports bar in Indian...,1.0,3.0,1.0,0.50,[Atmosphere was like a sports bar. Very clean ...


In [67]:
for i, row in response_synthetic_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n==============Error #{i+1}===============\n\n")
    print(f"Query:\n{row.query}\n")
    contexts = '\n\n'.join(row.contexts)
    print(f"Context:\n{contexts}\n")
    print(f"Answer:\n{row.answer}\n----\n")
    expected = response_synthetic_eval_dataset_dict.get(row.query)
    if not expected:
        logger.error(f"Could not find query {row.query} in synthetic_response_eval_dataset_dict!")
        continue
    expected_answer = expected['reference_answer']
    print(f"Expected Answer:\n{expected_answer}\n")
    expected_contexts = expected['reference_contexts']
    print(f"Expected Contexts:\n{expected_contexts}\n")



==============Error #1===============


Query:
- What are some pizza places known for their quality and flavor in White House, TN?

Context:
Domino's keep being you and I'll keep patronizing your fine establishment.

I hadn't had Dominos for at least 15+ years so when I moved to Whitehouse, we gave them a try.  I'm pretty sure that a frozen pizza would have tasted better!  Even a generic store brand one would have been better.  I ask for extra sauce but what I didn't know is that when I ask for extra sauce they must have misunderstood me and thought I said extra grease.  I had to use paper towels to soak up all that grease before I could even make an attempt to taste it & there was barely any sauce on it & what little bit there was, had no flavor.  The cheese tasted like it was not even real; the imitation stuff.  The pepperoni was dry & curled up around the edges where they were laying in all that grease I suppose.  I got through 1/2 a piece and the entire thing went in the trash, m

In [79]:
query = "- What are some pizza places known for their quality and flavor in White House, TN?"
response = query_engine.query(query)
display_response(response, source_length=1000, show_source=True, show_metadata=True, show_source_metadata=True)

Generated queries:
pizza places quality flavor White House TN
"I recently visited a pizza place in White House, TN, and I was blown away by the quality and flavor of their pies. The crust was perfectly crispy, and the toppings were fresh and delicious. Highly recommend checking it out!"


**`Final Response:`** Unfortunately, there isn't enough information to provide a definitive answer. However, it's worth noting that a single location with a reputation for quality and flavor doesn't necessarily mean that other locations in the same area will have the same reputation.


#### Referenced Paragraphs
**Domino's Pizza**

> Domino's keep being you and I'll keep patronizing your fine establishment. (Review ID: <REVIEW_ID>_8XEmtRXQ3g277EtXOstNA</REVIEW_ID>)

> I hadn't had Dominos for at least 15+ years so when I moved to Whitehouse, we gave them a try.  I'm pretty sure that a frozen pizza would have tasted better!  Even a generic store brand one would have been better.  I ask for extra sauce but what I didn't know is that when I ask for extra sauce they must have misunderstood me and thought I said extra grease.  I had to use paper towels to soak up all that grease before I could even make an attempt to taste it & there was barely any sauce on it & what little bit there was, had no flavor.  The cheese tasted like it was not even real; the imitation stuff.  The pepperoni was dry & curled up around the edges where they were laying in all that grease I suppose.  I got through 1/2 a piece and the entire thing went in the trash, my husband couldn't even eat it.  I just couldn't do it.  I won't even get into the price!  (Review ID: <REVIEW_ID>le30LFt0MY-DZTctfrJJKQ</REVIEW_ID>)

> When we do have delivery its fast and hot. Must have a really great team at this location! (Review ID: <REVIEW_ID>gn8_q0-x36PvNli6Rc_Upw</REVIEW_ID>)

> Should have gone with a $5 pizza that was cooked. (Review ID: <REVIEW_ID>Nd9IlLVwTdG0JEFYnA4cJw</REVIEW_ID>)

**Joe's Pizza**

> I've been visiting this pizza place since I was a kid, and the quality and taste are the same time after time. 5* on their consistency.  (Review ID: <REVIEW_ID>lgCDQrjybDW8BKrxt-Tt_A</REVIEW_ID>)

---

**`Source Node 1/5`**

**Node ID:** f4468a7d-858c-4187-886c-16b646ef6af5<br>**Similarity:** 1.6414785385131836<br>**Text:** Domino's keep being you and I'll keep patronizing your fine establishment.<br>**Metadata:** {'review_id': '_8XEmtRXQ3g277EtXOstNA', 'user_id': 'tPJ4QRnQ7T1rWd-CjI7DZQ', 'business_id': 'sqSqqLy0sN8n2IZrAbzidQ', 'review_stars': 3, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2021-12-16T02:10:31', 'biz_name': "Domino's Pizza", 'biz_address': '3001 Highway 31 W', 'biz_city': 'White House', 'biz_state': 'TN', 'biz_postal_code': '37188', 'biz_latitude': 36.4647467622, 'biz_longitude': -86.6591868908, 'biz_stars': 3.5, 'biz_review_count': 8, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': None, 'Ambience': None, 'BYOB': None, 'BestNights': None, 'BikeParking': None, 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': None, 'BusinessParking': None, 'ByAppointmentOnly': None, 'Caters': None, 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': None, 'GoodForMeal': None, 'HappyHour': None, 'HasTV': None, 'Music': None, 'NoiseLevel': None, 'OutdoorSeating': None, 'RestaurantsAttire': None, 'RestaurantsDelivery': None, 'RestaurantsGoodForGroups': None, 'RestaurantsPriceRange2': '1', 'RestaurantsReservations': None, 'RestaurantsTableService': None, 'RestaurantsTakeOut': None, 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': None}, 'biz_categories': ['Pizza', 'Chicken Wings', 'Sandwiches', 'Restaurants'], 'biz_hours': {'Friday': '10:0-1:0', 'Monday': '10:0-0:0', 'Saturday': '10:0-1:0', 'Sunday': '10:0-0:0', 'Thursday': '10:0-0:0', 'Tuesday': '10:0-0:0', 'Wednesday': '10:0-0:0'}}<br>

...

**Node ID:** 82c36381-1c5a-4126-8087-9286b18da128<br>**Similarity:** 1.1970523595809937<br>**Text:** Should have gone with a $5 pizza that was cooked.<br>**Metadata:** {'review_id': 'Nd9IlLVwTdG0JEFYnA4cJw', 'user_id': '_GZUfylSfVj2gNOkiqsSoQ', 'business_id': 'sqSqqLy0sN8n2IZrAbzidQ', 'review_stars': 1, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-08-26T14:44:11', 'biz_name': "Domino's Pizza", 'biz_address': '3001 Highway 31 W', 'biz_city': 'White House', 'biz_state': 'TN', 'biz_postal_code': '37188', 'biz_latitude': 36.4647467622, 'biz_longitude': -86.6591868908, 'biz_stars': 3.5, 'biz_review_count': 8, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': None, 'Ambience': None, 'BYOB': None, 'BestNights': None, 'BikeParking': None, 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': None, 'BusinessParking': None, 'ByAppointmentOnly': None, 'Caters': None, 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': None, 'GoodForMeal': None, 'HappyHour': None, 'HasTV': None, 'Music': None, 'NoiseLevel': None, 'OutdoorSeating': None, 'RestaurantsAttire': None, 'RestaurantsDelivery': None, 'RestaurantsGoodForGroups': None, 'RestaurantsPriceRange2': '1', 'RestaurantsReservations': None, 'RestaurantsTableService': None, 'RestaurantsTakeOut': None, 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': None}, 'biz_categories': ['Pizza', 'Chicken Wings', 'Sandwiches', 'Restaurants'], 'biz_hours': {'Friday': '10:0-1:0', 'Monday': '10:0-0:0', 'Saturday': '10:0-1:0', 'Sunday': '10:0-0:0', 'Thursday': '10:0-0:0', 'Tuesday': '10:0-0:0', 'Wednesday': '10:0-0:0'}}<br>

---

**`Source Node 5/5`**

**Node ID:** 7f5d27a9-c35a-45d0-9810-e96122c4ebbb<br>**Similarity:** -6.092642307281494<br>**Text:** I've been visiting this pizza place since I was a kid, and the quality and taste are the same time after time. 5* on their consistency.<br>**Metadata:** {'review_id': 'lgCDQrjybDW8BKrxt-Tt_A', 'user_id': 'xfUV3e0TZLNaYVDw4NTZIw', 'business_id': 'JgpnXv_0XhV3SfbfB50nxw', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-01-23T19:27:30', 'biz_name': "Joe's Pizza", 'biz_address': '2038 N Dale Mabry Hwy', 'biz_city': 'Tampa', 'biz_state': 'FL', 'biz_postal_code': '33607', 'biz_latitude': 27.9605142, 'biz_longitude': -82.506127, 'biz_stars': 4.0, 'biz_review_count': 35, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'beer_and_wine'", 'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'False', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}", 'HappyHour': None, 'HasTV': 'True', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'False', 'RestaurantsAttire': "u'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '1', 'RestaurantsReservations': 'False', 'RestaurantsTableService': None, 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': "u'no'"}, 'biz_categories': ['Restaurants', 'Pizza'], 'biz_hours': {'Friday': '11:0-3:0', 'Monday': '11:0-22:0', 'Saturday': '11:0-3:0', 'Sunday': None, 'Thursday': '11:0-22:0', 'Tuesday': '11:0-22:0', 'Wednesday': '11:0-22:0'}}<br>

{'f4468a7d-858c-4187-886c-16b646ef6af5': {'review_id': '_8XEmtRXQ3g277EtXOstNA',
  'user_id': 'tPJ4QRnQ7T1rWd-CjI7DZQ',
  'business_id': 'sqSqqLy0sN8n2IZrAbzidQ',
  'review_stars': 3,
  'useful': 0,
  'funny': 0,
  'cool': 0,
  'date': '2021-12-16T02:10:31',
  'biz_name': "Domino's Pizza",
  'biz_address': '3001 Highway 31 W',
  'biz_city': 'White House',
  'biz_state': 'TN',
  'biz_postal_code': '37188',
  'biz_latitude': 36.4647467622,
  'biz_longitude': -86.6591868908,
  'biz_stars': 3.5,
  'biz_review_count': 8,
  'biz_is_open': 1,
  'biz_attributes': {'AcceptsInsurance': None,
   'Alcohol': None,
   'Ambience': None,
   'BYOB': None,
   'BestNights': None,
   'BikeParking': None,
   'BusinessAcceptsBitcoin': None,
   'BusinessAcceptsCreditCards': None,
   'BusinessParking': None,
   'ByAppointmentOnly': None,
   'Caters': None,
   'CoatCheck': None,
   'Corkage': None,
   'DogsAllowed': None,
   'DriveThru': None,
   'GoodForDancing': None,
   'GoodForKids': None,
   'GoodForMeal'

### Manually Curated
Ref: https://docs.llamaindex.ai/en/stable/examples/llama_dataset/ragdataset_submission_template/#1c-creating-a-labelledragdataset-from-scratch-with-manually-constructed-examples

In [82]:
response_curated_eval_dataset = response_evaluator.generate_curated_dataset(cfg)

2024-08-10 16:41:29.411 | INFO     | src.run.eval.response:generate_curated_dataset:125 - Persisting curated response eval dataset at data/020_add_context_relevancy_metric/response_curated_eval_dataset.json...


In [83]:
response_curated_eval_prediction_dataset = await response_curated_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=cfg.batch_size, show_progress=True
)

Batch processing of predictions:   0%|                                                                                                                                                                         | 0/1 [00:00<?, ?it/s]

Generated queries:
recommended restaurants Santa Barbara
"I recently visited Santa Barbara and dined at The Lark, which offers a fantastic farm-to-table experience. The ambiance is great, and the food is delicious!"


Batch processing of predictions: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.53s/it]


In [84]:
response_curated_mean_scores_df, response_curated_deep_eval_df = response_evaluator.evaluate_labelled_rag_dataset(
    response_curated_eval_dataset,
    response_curated_eval_prediction_dataset,
    dataset_name="curated",
    judge_model=cfg.eval_cfg.response_eval_llm_model,
    cache_dp=cfg.notebook_cache_dp
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.07s/it]


In [85]:
response_curated_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,2.0
mean_relevancy_score,1.0
mean_faithfulness_score,1.0
mean_context_relevancy_score,0.5


In [86]:
response_curated_deep_eval_df

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
0,What are the recommended restaurants nearby Sa...,\nIt's always a good idea to explore local eat...,1.0,2.0,1.0,0.5,[Best meal we had in SB. Most places were a li...


#### Answers for target questions

In [87]:
from IPython.display import Markdown

In [88]:
from src.run.eval.manual_eval_dataset import MANUAL_EVAL_QA

In [89]:
for question, expected_answer in MANUAL_EVAL_QA:
    display(Markdown(f"\n\n### Question: {question}\n"))
    response = query_engine.query(question)
    display_response(response)



### Question: What are the recommended restaurants nearby Santa Barbara?


Generated queries:
recommended restaurants Santa Barbara nearby
"I recently dined at a fantastic restaurant near Santa Barbara called The Black Sheep. The ambiance was cozy, and the menu featured a great selection of local wines and farm-to-table dishes."


**`Final Response:`** It's always a good idea to explore local eateries when visiting a new city. If you're looking for a great place to grab a bite near the beach, consider checking out some of the casual, laid-back spots that serve up fresh, delicious food. Many of these places offer a variety of options, from breakfast sandwiches to pastries and coffee.


#### Referenced Paragraphs
**Helena Avenue Bakery**

> Finally some good food in downtown Santa Barbara.  Everything on the menu is great and could not recommend more.  (Review ID: <REVIEW_ID>8qeD6EhCxUEBj1dO4J4KfA</REVIEW_ID>)

> In Santa Barbara? This was the place to chill and relax for some refreshments and snacks.  (Review ID: <REVIEW_ID>-eNFOKe_VE4BCb6DYWabzg</REVIEW_ID>)

> If you're a tourist and spending the weekend in Santa Barbara, I'd recommend to give this place a go for sure. If you're a local, then not so much.  (Review ID: <REVIEW_ID>Mqh2anPpP_vvLtqCAk_1Xw</REVIEW_ID>)

> Another gem in Santa Barbara! Love the location, walking distance to the beach.  (Review ID: <REVIEW_ID>_S_Lnze72utd6e95FSnw6A</REVIEW_ID>)

> I ordered the breakfast sandwich, a side of potatoes and 2 danishes to go. The best food I ate my whole time in Santa Barbara. (Review ID: <REVIEW_ID>fo-FLKhR5HlKp60m880F4A</REVIEW_ID>)

### Error Analysis

In [90]:
response_curated_eval_dataset_dict = dict()
for example in response_curated_eval_dataset.examples:
    response_curated_eval_dataset_dict[example.query] = {
        "reference_answer": example.reference_answer,
        "reference_contexts": example.reference_contexts,
    }

In [91]:
response_curated_deep_eval_df.sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
0,What are the recommended restaurants nearby Sa...,\nIt's always a good idea to explore local eat...,1.0,2.0,1.0,0.5,[Best meal we had in SB. Most places were a li...


In [92]:
response_curated_eval_irrelevance_df = (
    response_curated_deep_eval_df
    .loc[lambda df: df['relevancy_score'].lt(1)]
    .sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])
)

for i, row in response_curated_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n==============Error #{i+1}===============\n\n")
    print(f"Query:\n{row.query}\n")
    contexts = '\n\n'.join(row.contexts)
    print(f"Context:\n{contexts}\n")
    print(f"Answer:\n{row.answer}\n----\n")
    expected = response_curated_eval_dataset_dict.get(row.query)
    if not expected:
        logger.error(f"Could not find query {row.query} in synthetic_response_eval_dataset_dict!")
        continue
    expected_answer = expected['reference_answer']
    print(f"Expected Answer:\n{expected_answer}\n")
    expected_contexts = expected['reference_contexts']
    print(f"Expected Contexts:\n{expected_contexts}\n")

# Persist run metadata

In [93]:
from src.run.utils import parse_collect_log, flatten_dict

In [94]:
collect_info = parse_collect_log(collect_fp)

In [95]:
if ARGS.LOG_TO_MLFLOW:
    import mlflow

    logger.info("Logging [COLLECT] info to MLflow...")
    mlflow.log_params(collect_info)
    logger.info("Logging config to MLflow...")
    mlflow.log_params(flatten_dict(cfg.model_dump(), "cfg", sep='.'))
    logger.info(f"Logging Retrieval Synthetic Eval Results to MLflow...")
    retrieval_evaluator.log_to_mlflow(cfg)
    logger.info(f"Logging Response Eval Results to MLflow...")
    try:
        response_evaluator.log_to_mlflow(
            cfg,
            'synthetic',
            response_synthetic_mean_scores_df,
            response_synthetic_deep_eval_df
        )
        response_evaluator.log_to_mlflow(
            cfg,
            'curated',
            response_curated_mean_scores_df,
            response_curated_deep_eval_df
        )
    except:
        ...

2024-08-10 16:41:42.465 | INFO     | __main__:<module>:4 - Logging [COLLECT] info to MLflow...
2024-08-10 16:41:42.488 | INFO     | __main__:<module>:6 - Logging config to MLflow...
/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
2024-08-10 16:41:42.530 | INFO     | __main__:<module>:8 - Logging Retrieval Synthetic Eval Results to MLflow...
2024-08-10 16:41:42.680 | INFO     | __main

# Clean up

In [96]:
if ARGS.LOG_TO_MLFLOW:
    mlflow.end_run()

# Archive